<a href="https://colab.research.google.com/github/beltima/extract-wikidata-names/blob/get-names-dictionary/Extract_Wiki_Data_names_try_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qwikidata
import qwikidata

In [2]:
from qwikidata.sparql import get_subclasses_of_item
from qwikidata.sparql import return_sparql_query_results

In [3]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def find_fname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  res = return_sparql_query_results(find_fn_id_query)
  id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  return id

In [4]:
def find_lname_id(name): 
  find_ln_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE {{
  ?item rdfs:label "{name}"@en;
        wdt:P31 wd:Q101352.
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}

'''
  res = return_sparql_query_results(find_ln_id_query)
  id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  return id #Q101352 last name id

In [6]:
def find_similar_from_id(id1): 
  #find similar names from the ID just found for the target name
  id = id1
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  res = return_sparql_query_results(find_similar_query)
  similar_name_list = get_itemlabel(res)
  return similar_name_list

In [34]:
# print(find_lname_id('Baker'))
# print(find_fname_id('Joshua'))
# print(find_similar_from_id('Q111549586'))
# print(find_similar_from_id('Q3284319'))


['Ernst', 'Ernest']


In [14]:
def similar_full_names(name): 
  f_name, l_name = name.split(' ')

  f_name_id = find_fname_id(f_name)
  l_name_id = find_lname_id(l_name)

  fname_similar = find_similar_from_id(f_name_id)
  lname_similar = find_similar_from_id(l_name_id)

  fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
  lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]

  similar_full_names = fname_list + lname_list
  pair = {name: similar_full_names}
  return pair

In [15]:
name = 'Joshua Baker'
similar_full_names(name)

{'Joshua Baker': ['Joshuah Baker',
  'Josh Baker',
  'Jozue Baker',
  'Josua Baker',
  'Joosua Baker',
  'Yosua Baker',
  'Josuah Baker',
  'Giosuè Baker',
  'Yehoshua Baker',
  'Joshua Bäcker',
  'Joshua Bakker',
  'Joshua Becker',
  'Joshua Boulanger',
  'Joshua Boulenger']}

In [31]:
name1 = 'Ernst Albrecht'
similar_full_names(name1)

{'Ernst Albrecht': ['Ernestan Albrecht',
  'Ernest Albrecht',
  'Ernesto Albrecht',
  'Arnošt Albrecht',
  'Ernő Albrecht',
  'Q111549586 Albrecht',
  'Ernst Abel',
  'Ernst Albert',
  'Ernst Alberto',
  'Ernst Avraham',
  'Ernst Alberts',
  'Ernst Albertus',
  'Ernst Albracht',
  'Ernst Adelbrecht',
  'Ernst Albercht',
  'Ernst Allbrecht']}

In [17]:
male_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581097.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#male Q6581097

female_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581072.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#female Q6581072

In [19]:
def Merge(dict1, dict2):
  return(dict2.update(dict1))

In [27]:
male_politicans = [ x["itemLabel"]["value"] for x in return_sparql_query_results(male_politicians_query)['results']['bindings']]
female_politicans = [ x["itemLabel"]["value"] for x in return_sparql_query_results(female_politicians_query)['results']['bindings']]

In [28]:
print(male_politicans)
print(female_politicans)

['Heinrich Scheuch', 'Hermann Muhs', 'Ernst Albrecht', 'Konrad Wolf', 'Peter Schmidhuber', 'Dietrich Stobbe', 'Alfred Herrhausen', 'Heinrich Held', 'Walter Blume', 'Heinz Auerswald']
['Zahra Mohammadzadeh', 'Yukiko Kada', 'Oriana Fallaci', 'Anniken Huitfeldt', 'Marianne Aasen', 'Kirron Kher', 'Angela Davis', 'Rosario Green', 'Valeria Ciavatta', 'Mary Carey']


In [39]:
for name in male_politicans: 
  print(similar_full_names(name))

JSONDecodeError: ignored